# Introduction + Setup

Hi everyone. My name is Michael, and I'm a master's student at UCSD studying machine learning and data science. I've been working with the UCSD Library's Data Curation team for the last 8 months to build a data collection tool that scrapes APIs and websites. This workshop is going to take you all through the first steps that I went through in order to collect API data in a robust and reproducible way.

To start off, we're assuming that everyone here has at least a basic understanding of what an API is, and is comfortable programming in Python. Since this is a workshop, we encourage you all to follow along on your own computer. If you don't have jupyter or another Python environment set up, you can use [Google Colab](https://colab.research.google.com) to write your code. I'll paste a link to that in the chat, and we can wait a minute or so in case anyone wants to get that loaded before we get started. Once you're ready to go, or if you're not planning on coding with us, go ahead and click on the green checkmark in the reactions tab so we know when everyone is ready. If anyone has any questions, you can ask in the chat. If you're more comfortable sending it privately, you can send it to Dennis, Kat, or Jamie, and they'll make sure that I get it answered.

While everyone is getting their environments ready, there is an optional package I'll be using to better understand the structure of our query results, which is the ```flatten_json``` package. This can be installed by 

```bash
pip install flatten_json
```

We'll also be making use of ```pandas```. If you're working with Python through Google Colab or an Anaconda installation, it should already be installed, but if you need to download it,

```bash
pip install pandas
```

# Background Information

Today we'll be going through the full process of requesting the data we want from a REST API. For this workshop, we're focusing on Zenodo. Zenodo is an online Open Science research repository, which houses close to two and a half million publications, datasets, codebases, and other research related items. This makes it a great API to explore because there is a ton of data that we can collect. 

[Zenodo](https://zenodo.org)

Before we get started with any API, we need to check out the documentation to get familiar with the querying process. A lot of API's that you run into will work pretty similarly, but we still need to get the specifics.

[Zenodo API](https://developers.zenodo.org)

Taking a look at the documentation, there are a few key points to take note of.
First, we see that Zenodo uses a REST API, which means that we're going to be wanting to use the ```requests``` library in python to make our queries.

Second, we see that there's a section on authenticating our requests over HTTPS. Although the section outlines that the requests will fail if they're not authenticated, we don't actually need to go through the process of creating a personal token and authenticating for simple data queries to Zenodo. I'm not sure why, but it makes this demo a bit less cumbersome.

Third, we see that the base url for our requests is given (```https://zenodo.org/api/```), so we need to take note of that for later.

Continuing through, we see the standards that our responses will be JSON objects and successful responses will have a status_code of 200.

Now for the actual queries, we want to be searching over the records. This section tells us that the query url is going to be ```'https://zenodo.org/api/records'```, and it gives us some insight on the different parameters that we can use to perform a search. Now for this application, let's say that we're interested in collecting all of the records that Zenodo has on basketball. Since we want to collect all of the records, we won't need to worry about a lot of the parameters that restrict the search. Instead, we'll mainly be focusing on 

- q: The Elasticsearch query string search query
- page: Page number of results
- size: The number of result per page.

This means that the results are paginated, which is common for APIs that could potentially return a lot of data. If our query had a few hundred thousands results, returning all of them in one set wouldn't be feasible.

Now that we've got a rough idea of how we can work with the API, let's dive into it.

# Importing the necessary librarys

There are a couple of different librarys that we're going to be using, so let's start out importing them all at the top in order to keep the full code a bit cleaner.

1. ```requests``` 
    Since we're working with a REST API, we're going to want to use the requests library in order to send our requests to the API.
    
2. ```pandas```
    Since we're making a handful of requests and need to store them, we're going to use Pandas DataFrames to keep all of our responses in one place.
    
3. ```flatten_json```
    We saw that the data is returned in a JSON format. It'll be helpful to be able to view the "nested" JSON data in a flatter form.

In [ ]:
import requests
import pandas as pd
from flatten_json import flatten

# Making Requests

As a refresher, let's take a look at how we make a request in general

In [ ]:
# r = requests.get(url, params, headers)
r = requests.get("http://www.example.com/")
r

When we call the request.get module, it returns the response object, which is represented by the reponse code. In this case we have a 200 response, which means the request was successful. 

Next, let's break down what the full result contains. Python has a neat function called ```vars()``` which lets us see all the dictionary representation of an object.

In [ ]:
vars(r)

Now that we've got an idea about how these requests are formatted and how to make a request to Zenodo, let's try it out. 

## General Zenodo Requests

In [ ]:
r = requests.get("https://zenodo.org/api/records")

So recall that the documentation mentioned that the response is given in JSON format, so let's take a look at what it gives.

In [ ]:
output = r.json()
output.keys()

We can see that the output is structured into "aggregations", "hits", and "links", so let's take a look at what each of those are.

In [ ]:
output['aggregations']

In [ ]:
output['hits']

In [ ]:
output['links']

At first glance, it seems like aggregations is giving us some high level statistics about the breakdown of our query. Since we didn't make any specifications for search terms or time periods, it's reporting on all 2.3 million Zenodo records. The links data is giving some information on the current query and the next potential query page. And lastly, there's the hits data, which seems like what we really want. Although it looks like this is nested, so let's take a look at the breakdown of that.

In [ ]:
output['hits'].keys()

In [ ]:
output['hits']['total']

In [ ]:
output['hits']['hits']

So it looks like the total attribute of the hits is more high level information about the query, and the hits attribute contains the information that we're after. This looks pretty messy though, so we'll clean it up a bit by putting it into a DataFrame, which is easy to do since the JSON representations are stored as dictionarys.

In [ ]:
df = pd.DataFrame(output['hits']['hits'])
df.head()

Looking at the DataFrame, there's even more JSON nested in the hits data. This is getting kind of twisted, so let's just flatten the data and take a look at that to see what we're really getting back. First, let's take a look at the ```flatten_json``` functionality.

In [ ]:
flatten_json.flatten?

So the ```flatten``` function takes a dictionary, but we're dealing with a list of dictionarys, and we need to flatten all of them.

In [ ]:
flattened_hits = [flatten(json_dict) for json_dict in output['hits']['hits']]

That's actually going to come in handy a bunch, so we'll functionize it.

In [ ]:
def flatten_list(dicts):
    """Flatten iterable of nested dictionaries."""
    # Need to make sure we don't accidentally pass in the wrong data
    assert all([isinstance(d, dict) for d in dicts])
    
    return [flatten(d) for d in dicts]

In [ ]:
df = pd.DataFrame(flattened_hits)
df.head()

That's much more manageable to read, but now our DataFrame has 153 columns instead of just 12. This can be handy if we need all of the data in a flat format, but definitely be careful when you're unnesting content that can balloon up like that. 

## Specific Zenodo Requests

Now that we've seen how the general format of Zenodo requests are, we can finally move forward with getting all of the specific basketball data that we want.

In order to make the request, we have to pass those parameters to the ```params``` argument in the ```requests.get()``` function, which takes a dictionary. Let's set it up now so that we can continue to reference it.

In [ ]:
zenodo_records_url = 'https://zenodo.org/api/records'

In [ ]:
search_params = {'q': 'basketball'}

In [ ]:
r = requests.get(url = zenodo_records_url, params=search_params)

In [ ]:
output = r.json()
hits = output['hits']['hits']

In [ ]:
df = pd.DataFrame(hits)
df.head()

In [ ]:
df.shape

And now we can see that we're getting out basketball results back. Although we're only getting 10 results, so we need to use the pagination parameters that we read about earlier. To make this easier, let's put this into a function too.

First, let's quickly check the assumptions that we're making so that our function doesn't crash. We're assuming that we're going to search for a string, since that's what Zenodo requires. We're assuming that we get a proper 200 status code. We're also assuming that the results are given in the ```r.json()``` nested hits hits item. Is this always going to happen?

In [ ]:
r = requests.get(zenodo_records_url, params={'q': 'fjkdajfeiowajtio'})

In [ ]:
r.json()

Now we see that ```output['hits']['hits']``` doesn't exist. so we need to be careful about just indexing the output dictionary when it might be empty. If we aren't sure that a dictionary has a key or not, we can use the ```dict.get()``` function to be safe. By default, an improper key will return None.

In [ ]:
def get_zenodo_search_output(search_term, page, size):
    """Return the output for a Zenodo record query.
    
    Parameters
    ----------
    search_term : str
    page : int
    size : int
    
    Returns
    -------
    output_df : pandas.DataFrame
    """
    
    assert isinstance(search_term, str)
    assert isinstance(page, int)
    assert isinstance(size, int)
    
    search_url = 'https://zenodo.org/api/records'
    search_params = {'q': search_term, 'page': page, 'size': size}
    
    r = requests.get(search_url, params=search_params)
    output = r.json()
    
    if r.status_code == 200 and output.get('hits').get('hits'):
        output_df = pd.DataFrame(output['hits']['hits'])
    else:
        return r
    
    return output_df

In [ ]:
get_zenodo_search_output('basketball', 1, 20)

Awesome, now we're able to get more results and only call one line of code to do it. Let's test it out.

In [ ]:
output = get_zenodo_search_output('basketball', 100, 1000)
output

Now we see that we're getting a 400 response, so let's check it out.

In [ ]:
vars(output)

So we can't search for more than 10,000 results at a time.

In [ ]:
requests.get(zenodo_records_url, params={'q': 'basketball'}).json()['hits']['total']

Luckily we won't run into any 10,000 limit issues, but if we have time, let's try to figure out how we could get around this.

## Getting More Results

In order to get all of the records, we need to search over every page. Since we saw that there's a ```link``` attribute returned, let's continue to search that next link as long as it exists. In order to see what behavior to expect when we run out of pages, let's search the end results (which we can do since we know how many total records there are.

In [ ]:
r = requests.get(zenodo_records_url, params={'q': 'basketball', 'page': 4, 'size': 100})
output = r.json()

In [ ]:
output['links']

When there's no more pages, they just don't return a ```next``` link. So a potential loop for this could be something like
```python
# Perform initial search
r = requests.get(zenodo_search_url, params=search_params)
output = r.json()

if not r.status_code == 200 and output.get('hits').get('hits'):
    return None

cumulative_df = pd.DataFrame()

while r.status_code == 200 and output['links'].get('next'):
    # Get hits output
    hits_df = pd.DataFrame(output['hits']['hits'])
    
    # Append results to df
    cumulative_df = pd.concat([cumulative_df, hits_df]).reset_index(drop=True)
    
    # Get new results
    r = requests.get(zenodo_search_url, params=search_params)
    output = r.json()

return cumulative_df
```

In [ ]:
def get_zenodo_search_output(search_term):
    """Return the output for a Zenodo record query.
    
    Parameters
    ----------
    search_term : str
    
    Returns
    -------
    cumulative_df : pandas.DataFrame
    """
    
    assert isinstance(search_term, str)
    
    search_url = 'https://zenodo.org/api/records'
    print('first search')
    search_params = {'q': search_term, 'size': 100} # Perform initial search
    print('finished first search')
    
    r = requests.get(search_url, params=search_params)
    output = r.json()

    if not r.status_code == 200 and output.get('hits').get('hits'):
        return None

    cumulative_df = pd.DataFrame()

    while r.status_code == 200 and output['links'].get('next'):
        print('searching...')
        # Get hits output
        hits_df = pd.DataFrame(output['hits']['hits'])

        # Append results to df
        cumulative_df = pd.concat([cumulative_df, hits_df]).reset_index(drop=True)

        # Get new results
        r = requests.get(output['links'].get('next'))
        output = r.json()
    
    hits_df = pd.DataFrame(output['hits']['hits'])
    cumulative_df = pd.concat([cumulative_df, hits_df])

    return cumulative_df

In [ ]:
get_zenodo_search_output('basketball')

Now we see that we're getting all of the results we wanted, so our job here is done. Let's take a look at some of the challenges that we could run into when using this API, which also often come up for other API's.

- Cirmumventing 10,000 record maximum
    Say we want to get all of the records for a term that has more than 10,000 results. The API is going to limit us, but surely there has to be some way around that. This is a somewhat common restriction with APIs, and it's caused me quite a bit of headache. I wanted to retrieve all 34,000 "machine learning" results from Figshare, which is another research repository, but they have a similar restriction. While you can't always beat the limit, let's do some brain-storming about how to see if it would be possible.